# Вступление
Цель - создать набор данных и простую (1 скрытый слой) сеть для целей классификации. Я хочу, чтобы все было как можно более замкнутым. Возможно, есть более гибкие конструкции, но я сосредоточен только на решении классификатора цифр MNIST.

Прежде всего, полезный импорт.

In [10]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

## 1. Создайем загрузчики данных
Самым сложным здесь было получение правильных форм и типов. Есть 2 объекта: один для обучающего набора (который возвращает данные и истинную метку) и один для тестового набора (который возвращает только данные).
Поскольку в этой простой модели нас не интересует форма, данные возвращаются в виде вектора длиной 784.

In [11]:
class KaggleMNIST(Dataset):
    """Пользовательский набор данных для использования с pytorch.
     Даже если у pytorch есть набор данных MNIST, для участия в Kaggle
     конкуренция, нам лучше использовать набор данных Kaggle.
     Этот класс загружает только набор данных поезда (изображения + метки).
    """
    
    def __init__(self, path):
        data = np.loadtxt(path + 'train.csv', delimiter=',', skiprows=1, dtype=np.float32)
        self._digits = torch.from_numpy(data[:, 1:]) / 255
        self._labels = torch.from_numpy(data[:, 0]).type(torch.long)
        self._size = len(data)
        print('Training dataset with MNIST digits loaded.')
        print('  Digits has shape:', self._digits.shape)
        print('  Labels has shape:', self._labels.shape)
        
    def __getitem__(self, idx):
        return (self._digits[idx], self._labels[idx])
    
    def __len__(self):
        return self._size
    
class KaggleMNIST_test(Dataset):
    """Пользовательский набор данных для использования с pytorch.
     Даже если у pytorch есть набор данных MNIST, для участия в Kaggle
     конкуренция, нам лучше использовать набор данных Kaggle.
     Этот класс загружает только тестовый набор данных (изображения).
    """
    
    def __init__(self, path):
        data = np.loadtxt(path + 'test.csv', delimiter=',', skiprows=1, dtype=np.float32)
        self._digits = torch.from_numpy(data) / 256
        self._size = len(data)
        print('Testing dataset with MNIST digits loaded.')
        print('  Digits has shape:', self._digits.shape)
        
    def __getitem__(self, idx):
        return self._digits[idx]
    
    def __len__(self):
        return self._size

## 2. Вспомогательная функция
Цель этой функции - уменьшить беспорядок.

In [12]:
def create_samplers(size, train_prop):
    """Функция, которая создает 2 подмножества из набора обучающих данных, одно из которых
     используйте din-обучение, а другое - для проверки.
    
     Параметры
     ----------
     размер: числовой, целочисленный.
         Количество элементов в наборе данных, которые нужно разделить на набор поездов и
         набор для проверки.
     train_prop: Числовое, с плавающей запятой.
         Число от 0 до 1, определяющее пропорцию элементов.
         который будет включен в набор для проверки.

     Return
     -------
     Кортеж с двумя объектами SubsetRandomSampler, первый из которых будет использоваться с
     обучающий DataLoader, а второй - с проверкой DataLoader.
    """
    cut_point = int(size * train_prop)
    shuffled = np.random.permutation(size)
    train_sampler = SubsetRandomSampler(shuffled[:cut_point])
    validation_sampler = SubsetRandomSampler(shuffled[cut_point:])
    return train_sampler, validation_sampler

## 3. Модель
Для этой задачи функцией потерь является кросс-энтропия, а оптимизация выполняется с помощью стохастического градиентного спуска. Скрытые слои имеют функцию активации LeakyReLU, но я не видел, чтобы она была лучше или хуже, чем простой ReLU. Использование signoid показало несколько худшие результаты. Больше ничего не пробовал.
Модель принимает заряжающие, а точность встроена.
Проверка - это частный метод, поскольку он используется во время обучения, а результаты (эволюция потерь и точности) выводятся на экран и возвращаются методом fit () в случае, если пользователь хочет что-то сделать.

In [13]:
class MLP_1_HL_Classification(nn.Module):
    """Простой многослойный перцептрон с одним скрытым слоем, использующий
     LeakyReLU как функция активации и кросс-энтропия как функция потерь,
     быть примененным к задачам классификации.
     Первая попытка поместить все в объект, поэтому модель
     самодостаточный. Не уверен, действительно ли это необходимо.
    """

    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size
        self.train_loader = None
        self.validation_loader = None
        self._optimizer_fn = torch.optim.SGD
        self._loss_fn = F.cross_entropy
        self._net = nn.Sequential(
            nn.Linear(self._input_size, self._hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self._hidden_size, self._output_size))
        
    def forward(self, batch):
        return self._net(batch)
    
    def define_loaders(self, train, validation):
        self.train_loader = train
        self.validation_loader = validation
            
    def _accuracy(self, outputs, labels):
        preds = torch.max(outputs, dim=1)[1]
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))
                
    def _validation_with_batch(self, batch):
        images, labels = batch 
        preds = self(images)
        loss = self._loss_fn(preds, labels, reduction='sum')
        acc = self._accuracy(preds, labels) * len(labels)
        return {'loss': loss, 'accuracy': acc}
        
    def _evaluate(self):
        with torch.no_grad():
            outputs = [self._validation_with_batch(batch) for batch in self.validation_loader]
        samples = len(self.validation_loader.sampler.indices)
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).sum() / samples
        batch_accuracies = [x['accuracy'] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracies).sum() / samples
        return {'loss': epoch_loss.item(), 'accuracy': epoch_accuracy.item()}
    
    def fit(self, epochs, learning_rate):
        print(f'Training the model for {epochs} epochs with learning rate {learning_rate}.')
        optim = self._optimizer_fn(self.parameters(), learning_rate)
        history = []
        for epoch in range(epochs):
            # Training Phase 
            for batch in self.train_loader:
                images, labels = batch 
                loss = self._loss_fn(self(images), labels)
                loss.backward()
                optim.step()
                optim.zero_grad()
            # Оценка эпох с помощью набора данных проверки.
            result = self._evaluate()
            print("Epoch [{}], loss: {:.4f}, accuracy: {:.4f}".format(epoch, result['loss'], result['accuracy']))
            history.append(result)
        return history  


## 4. Загрузка данных

In [14]:
train_set = KaggleMNIST('../input/digit-recognizer/')
test_set = KaggleMNIST_test('../input/digit-recognizer/')

Training dataset with MNIST digits loaded.
  Digits has shape: torch.Size([42000, 784])
  Labels has shape: torch.Size([42000])
Testing dataset with MNIST digits loaded.
  Digits has shape: torch.Size([28000, 784])


## 5. Дальнейшая настройка

In [15]:
batch_size = 80
hidden_layer = 500

train_sampler, validation_sampler = create_samplers(len(train_set), 0.8)

print(f'Size of training set: {len(train_sampler.indices)}.')
print(f'Size of validation set: {len(validation_sampler.indices)}.')

train_loader = DataLoader(train_set, batch_size, sampler = train_sampler)
validation_loader = DataLoader(train_set, batch_size, sampler = validation_sampler)

Size of training set: 33600.
Size of validation set: 8400.


## 6. Создайте модель и обучите ее
Учитывая, что мы знаем набор данных, размеры входного и выходного слоев являются константами.

Метод подгонки предназначен для последовательных вызовов, изменяющих эпохи и скорость обучения. Показанные здесь были найдены методом проб и ошибок. Эта сеть, кажется, имеет максимальную точность 97%, и большее количество эпох или больший скрытый слой не улучшают ее, по крайней мере, в моих попытках.

In [16]:
model = MLP_1_HL_Classification(784, hidden_layer, 10)
model.define_loaders(train_loader, validation_loader)

model.fit(3, 0.15)
model.fit(3, 0.1)
model.fit(5, 0.05)
model.fit(4, 0.025)
model.fit(4, 0.01)
model.fit(4, 0.005)

result0 = model._evaluate()
print(result0)

Training the model for 3 epochs with learning rate 0.15.
Epoch [0], loss: 0.3052, accuracy: 0.9087
Epoch [1], loss: 0.2314, accuracy: 0.9332
Epoch [2], loss: 0.1859, accuracy: 0.9461
Training the model for 3 epochs with learning rate 0.1.
Epoch [0], loss: 0.1622, accuracy: 0.9527
Epoch [1], loss: 0.1485, accuracy: 0.9567
Epoch [2], loss: 0.1362, accuracy: 0.9607
Training the model for 5 epochs with learning rate 0.05.
Epoch [0], loss: 0.1306, accuracy: 0.9623
Epoch [1], loss: 0.1279, accuracy: 0.9627
Epoch [2], loss: 0.1213, accuracy: 0.9633
Epoch [3], loss: 0.1181, accuracy: 0.9655
Epoch [4], loss: 0.1167, accuracy: 0.9649
Training the model for 4 epochs with learning rate 0.025.
Epoch [0], loss: 0.1131, accuracy: 0.9662
Epoch [1], loss: 0.1115, accuracy: 0.9664
Epoch [2], loss: 0.1118, accuracy: 0.9655
Epoch [3], loss: 0.1099, accuracy: 0.9665
Training the model for 4 epochs with learning rate 0.01.
Epoch [0], loss: 0.1086, accuracy: 0.9668
Epoch [1], loss: 0.1078, accuracy: 0.9680
E

## 7. Конец

In [17]:
predictions = [[idx+1, torch.max(model(point), dim=1)[1].item()] for idx, point in enumerate(DataLoader(test_set))]
submission = pd.DataFrame(predictions, columns=['ImageId', 'Label'])
submission.to_csv("submission.csv", index=False)